## Пример использования Word2Vec

In [ ]:
from gensim.models import Word2Vec

In [ ]:
nrows = 5000000 # количество строк, которое берем из файла
sentences = [] # лист предложений, по которым строится модель

j = 0 # вспомогательная переменная, чтобы выйти из цикла при достижении nrows

# опция rb при открытии файла на чтение помогает избежать выхода из цикла при случайном появлении в файле опеределенных спецсимволов
with open('query-count-direct-2015-09.txt', 'rb') as f:
    for line in f:
        # берем только те фразы, в которых больше двух слов
        # берем первый столбцев в файле (файл вида "фраза tab количество показов": "смотреть онлайн \t 564732")
        if len(line.split('\t')[0].split(' ')) > 1:
            # разбиваем поисковые фразы на слова и добавляем в лист sentences
            sentences.append(line.split('\t')[0].split(' '))
       
        j += 1
        # каждый миллион строк выводим ход процесса
        if j%1000000==0:
            print j
            
        # когда загрузили nrows строк выходим из цикла
        if j>nrows:
            break

In [ ]:
# смотрим сколько строк выгрузили
len(sentences)

In [ ]:
# параметры модели Word2Vec
num_features = 512    # размерность пространства векторов (чем больше, тем более точная модель)                      
min_word_count = 5   # учитываем в модели только слова, которые встретились более, чем min_word_count раз                        
num_workers = 256       # сколько процессов одновременно будут строить модель
context = 10          # сколько слов в контексте следует учитывать алгоритму
downsampling = 1e-3   # исключаем очень часто встречающиеся слова

In [ ]:
%%time

model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

In [ ]:
# можно сохранить модель на диске, чтобы в следующий раз не строить ее заново
# в качестве названия обычно записывают параметры, на которых она была построена
# загрузить модель можно с помощью функции .load
model.save('w2v_nf512_mwc5_nw256_ds1e3')

In [ ]:
# список слов, которые в итоге попали в модель
for word in model.vocab.keys():
    print word

In [ ]:
# выводим самые близкие в поисковых фразах слова к слову 'айфон'
for word, cosine in model.most_similar(positive=['айфон'], topn=10):
    print word, cosine

In [ ]:
# 'сша + доллар - россия'
for word, cosine in model.most_similar(positive=['сша', 'доллар'], negative=['россия'], topn=10):
    print word, cosine